In [ ]:
!pip install langchain
!pip install langgraph
!pip install -qU langchain[google-genai]
!pip install allosaurus

In [ ]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
chat_prompt_template = ChatPromptTemplate.from_messages(
    [("system", "Act as an expert Chinese language tutor with over 10 years of experience. Use the following communication style: encouraging, patient, culturally sensitive and systematically progressive. Gently correct mistakes (including pronounciation mistakes) in real time. Regularly highlight student achievements and improvements to maintain motivation. You are tutoring a native US English speaker. When speaking Chinese, never use vocabulary about the pre-2021 HSK-3 level under any circumstances.",), MessagesPlaceholder(variable_name="messages"),]
)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = chat_prompt_template.invoke(state["messages"])
    response = model.invoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#Adding Memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "CM"}}
while True:
  user_input = input("You>:")
  input_messages = [HumanMessage(user_input)]
  output = app.invoke({"messages": input_messages}, config)
  print("Teacher>:", end="")
  output["messages"][-1].pretty_print()